# TP1: análisis exploratorio

## Definiciones de funciones generales, imports, y carga de datos

In [ ]:
# magic function para hacer que los graficos de matplotlib se renderizen en el notebook.

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

plt.style.use('bmh') # haciendo los graficos un poco mas bonitos xD
plt.rcParams['figure.figsize'] = (18, 10)
%matplotlib inline

In [ ]:
def show_and_save_plot(title,xlabel='', ylabel='', **kwargs):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.savefig("./imgs/" + title + '.svg', pad_inches=0.3, bbox_inches='tight',**kwargs)
    plt.show()
    plt.close()

In [ ]:
# https://stackoverflow.com/a/50558529\n,
def __my_flatten_cols(self, how="_".join, reset_index=True):
    how = (lambda iter: list(iter)[-1]) if how == "last" else how
    self.columns = [how(filter(None, map(str, levels))) for levels in self.columns.values] \
                    if isinstance(self.columns, pd.MultiIndex) else self.columns
    return self.reset_index() if reset_index else self
pd.DataFrame.my_flatten_cols = __my_flatten_cols

In [ ]:
def event_grouper(event_list):
    result = []
    for event_name in event_list:
        def event_count_by_type(event_type, serie):
            return int(serie[serie == event_type].count())
        function = partial(event_count_by_type, event_name)
        function.__name__ = event_name
        result.append(function)
    return result

In [ ]:
raw_events = pd.read_csv('data/events.csv').sort_values('timestamp')

## Preprocesamiento

In [ ]:
brands = {'iphone':'apple', 'samsung':'samsung', 'lg':'lg', 'motorola':'motorola', \
            'sony':'sony', 'asus':'asus', 'quantum':'quantum', 'ipad':'apple', 'lenovo':'lenovo', \
             'blackberry' : 'blackberry'}
event_values = ['lead', 'conversion','staticpage','checkout','search engine hit', 'generic listing', 'ad campaign hit', 'visited site', 'brand listing', 'viewed product']
def findBrand(x):
    if(pd.isna(x)):
        return np.nan
    for key,value in brands.items():
        if(key in x.lower()):
            return value
    return np.nan
    

In [ ]:
raw_events['brand'] = raw_events['model'].map(findBrand)

# Funciones de limpieza de datos/creacion de features

In [ ]:
raw_events.timestamp = raw_events.timestamp.map(pd.Timestamp)
raw_events.time = raw_events.timestamp.map(lambda it: it.hour)
def megapixel_count(resolution):
    if type(resolution) != str: return np.NaN
    x_pixels, y_pixels = map(int, resolution.split("x"))
    return x_pixels*y_pixels/1000000
raw_events.screen_megapixels = raw_events.screen_resolution.map(megapixel_count)

# Introduccion a los datos

In [ ]:
raw_events.event.value_counts().plot(kind='barh',logx=True)
show_and_save_plot('Cantidad de tipos de eventos','Cantidad por evento', 'Tipo de evento')

In [ ]:
# Voy a intentar ver que proporcion de cada columna es nula
# Esto debería tener relación con que tipos de eventos tienen que atributos.
# Probablemente esta informacion salga de una db con un mapeo single_table
def myIsNa(serie):
    return serie.isna().sum() / len(serie.isna())
serie = pd.Series([1, 2, 3, np.NaN])
serie.isna().count()
grouped = raw_events.groupby('event').agg(myIsNa)
sns.heatmap(grouped, cbar=False)
show_and_save_plot('Columnas nulas por evento', 
                   xlabel='Columna', ylabel='Evento')
del grouped

# Exploracion de los distintos tipos de eventos por separado

## Eventos de busqueda

In [ ]:
# Podemos ver que la columna 'sku', se corresponde con los tipos de evento relacionados a un solo dispositivo
# Por otro lado, la columna 'skus', tiene un string que se corresponde con un listado de dispositivos.
busquedas = raw_events[raw_events.event == 'searched products'][['skus','timestamp','person','search_term']]
# verifico que para el mismo usuario y search_term hay resultados distintos, para validar lo anterior
busquedas[(busquedas.search_term.isna()) & (busquedas.person == '00aaa92e')]
del busquedas

#### Motores de busqueda mas usados

In [ ]:
raw_events['search_engine'].dropna().unique()

In [ ]:
raw_events['search_engine'].dropna().value_counts().plot(kind = 'bar')
show_and_save_plot('Motores de busqueda', xlabel='Motores', ylabel='Cantidad de busquedas')

#### Veo cuales son los modelos que mas se compraron a traves de los distintos motores de busqueda.

In [ ]:
search_events = raw_events[raw_events.event == 'ad campaign hit'].drop_duplicates(subset = 'person')
convertion_events = raw_events[raw_events.event == 'conversion']

In [ ]:
search_and_buy = convertion_events.merge(search_events, how = 'inner', on = 'person')

In [ ]:
len(search_and_buy)

In [ ]:
search_and_buy['brand_x'].value_counts().plot(kind = 'barh')
show_and_save_plot('Marcas mas buscadas y compradas a traves de motor de busqueda' )

In [ ]:
raw_events[raw_events.event == 'conversion']['brand'].value_counts().plot(kind = 'barh')
show_and_save_plot('Marcas mas compradas', xlabel='Cantidad compras efectuadas', ylabel='Marcas')

## Analisis de eventos por marcas

In [ ]:
brand_conversion = raw_events[raw_events['event'] == 'conversion'][['brand','timestamp']]

In [ ]:
brand_conversion['date'] = brand_conversion['timestamp'].map(lambda x: str(x.year) + "-" + str(x.month))

In [ ]:
brand_conversion = brand_conversion.set_index('date')
brand_conversion.groupby(level='date').brand.value_counts().unstack('brand').plot.bar(stacked=True, width = 1)
show_and_save_plot('Distribucion ventas por mes y marca', xlabel='Fecha', ylabel='Cantidad')

## Eventos de visited_site

In [ ]:
raw_events.channel.value_counts().plot(kind='barh',logx=True)
show_and_save_plot('Cantidad de tipos de canales', xlabel='Cantidad por canal',
                    ylabel='Tipo de canal')

In [ ]:
raw_events.device_type.value_counts().plot(kind='barh',logx=True)
show_and_save_plot('Cantidad de tipos de dispositivos', xlabel='Cantidad por tipo de dispositivo', ylabel='Tipo de dispositivo')

In [ ]:
# Vemos que las resoluciones de pantalla son en general bajas, lo que da la idea
# de que la aplicación es usada por personas de bajo poder adquisitivo.
raw_events.screen_resolution.value_counts()[0:15].plot(kind='barh',logx=True)
show_and_save_plot('Cantidad por tipos de resoluciones', xlabel='Cantidad por tipo de resolución', 
                   ylabel='Tipo de resolución')

# Exploracion de todos los eventos en conjunto

In [ ]:
# Nos damos una idea de qué resoluciones de pantalla tienen los usuarios
raw_events.screen_megapixels.hist(bins=[x*0.3 for x in range(10)])
show_and_save_plot('Distribución de cantidad de pixeles de las pantallas', xlabel='Megapixeles',
                   ylabel='Cantidad de usuarios')

In [ ]:
raw_events.timestamp.hist()
show_and_save_plot("distribución de eventos en el tiempo", xlabel='Fecha',
                   ylabel='Cantidad acumulada de eventos')

In [ ]:
# Cuantos 'skus' distintos hay
print( "hay {} dispositivos y {} dispositivos distintos".format(len(raw_events.skus.dropna().values), len(raw_events.skus.dropna().unique())))

In [ ]:
# Hora del dia en la que se producen los eventos:
raw_events.time.hist(bins=24)
show_and_save_plot('Distribucion de eventos en las horas del dia', xlabel='Hora del dia',
                   ylabel='Cantidad de eventos')

## Relación entre eventos de un mismo usuario
Primero voy a ver cuantos eventos hay por cada usuario ¿es usual que haya cientos por cada uno? ¿Solo un par?

In [ ]:
def get_persons_data(raw_events):
    def first_event(series):
        return series.values[0]
    def last_event(series):
        return series.values[-1]
    persons = raw_events.groupby('person').agg({
        'timestamp':['min','max','count'],
        'event': event_grouper(event_values) + [first_event, last_event]
    })
    persons['time_spent'] = persons['timestamp','max'] - persons['timestamp','min']
    persons.time_spent = persons.time_spent.map(lambda it: it.total_seconds()/3600)
    return persons.my_flatten_cols().set_index('person')
def add_first_last_events(persons, raw_events):
    def get_last_and_first_events(row):
        events = raw_events[(raw_events.person == row.person) & (raw_events.timestamp == row.timestamp_min)]
        row['first_events'] = events.event.sort_values().str.cat(sep=', ')
        events = raw_events[(raw_events.person == row.person) & (raw_events.timestamp == row.timestamp_max)]
        row['last_events'] = events.event.sort_values().str.cat(sep=', ')
        return row
    return persons.apply(get_last_and_first_events, axis=1)

In [ ]:
persons = get_persons_data(raw_events)

In [ ]:
tempdf = pd.DataFrame()
tempdf['everyone'] = persons.timestamp_count
tempdf['buyers'] = persons[persons.event_conversion > 0].timestamp_count
tempdf.plot.box(showfliers=False)
show_and_save_plot('Cantidad de eventos general vs compradores', ylabel='cantidad')

In [ ]:
persons.reset_index()

In [ ]:
add_first_last_events(persons.sample(300).reset_index(), raw_events).first_events.value_counts().plot(kind='barh')
show_and_save_plot('Conjunto de primer evento',xlabel='cantidad',ylabel='conjunto de eventos')

In [ ]:
add_first_last_events(persons.sample(300).reset_index(), raw_events).last_events.value_counts().plot(kind='barh')
show_and_save_plot('Conjunto de ultimo evento',xlabel='cantidad',ylabel='conjunto de eventos')

In [ ]:
raw_events[raw_events.event == 'visited site']

In [ ]:
# Si muestro los outliers usa mucho espacio para los mismos, arriba del boxplot
persons.time_spent.plot(kind='box', showfliers=False)
show_and_save_plot('Tiempo que pasan los usuarios en el sitio',xlabel='Distribución', ylabel='Tiempo (horas)')

In [ ]:
raw_events[raw_events.person=='000a54b2'].event.values[-1]

In [ ]:
# NO hay relacion evidente entre el tiempo que se pasa en la plataforma y la cantidad de eventos que tiene. Ni siquiera es lineal.

for i in map(lambda it: 'event_'+it, event_values):
    persons.plot.scatter('time_spent', i, alpha=0.1)
plt.show()

In [ ]:
raw_events[raw_events.person == '000a54b2'].sort_values('timestamp')

In [ ]:
persons.event_first_event.value_counts().plot(kind='barh',logx=True)
show_and_save_plot('Distribucion del primer evento de un usuario(log)')

In [ ]:
persons.event_last_event.value_counts().plot(kind='barh', logx=True)
show_and_save_plot('Distribucion del ultimo evento de un usuario(log)')

In [ ]:
#En vista de que hay muchos usuarios cuyo ultimo evento es un checkout, me interesa si es el unico
print("hay ",
      (persons.event_last_event == 'checkout').size,
      "usuarios con ultimo evento checkout, y ",
      (persons.timestamp_count).size,
      "usuarios con un unico evento")

## Features de cada modelo

In [ ]:
def get_models_data(raw_events):
    def event_count_by_type(event_type, serie):
        return int(serie[serie == event_type].count())
    def viewed_product(it):
        return event_count_by_type('viewed product',it)
    def checkout(it):
        return event_count_by_type('checkout',it)
    def conversion(it):
        return event_count_by_type('conversion',it)
    def lead(it):
        return event_count_by_type('lead',it)
    event_functions = [ viewed_product, checkout, conversion, lead]
    models = raw_events\
        .groupby('model')\
        .agg({'timestamp':['count','min','max'],'event':event_functions})
    models['cantidad_eventos'] = models['timestamp','count']
    models = models.drop(labels=[('timestamp','count')],axis=1)
    models = models[models.cantidad_eventos > 100]
    for i in ['checkout', 'conversion','lead','viewed_product']:
        models['event',i] = models['event',i]/models['cantidad_eventos']
    return models

In [ ]:
models = get_models_data(raw_events).sort_values('cantidad_eventos', ascending=False)
models.event.head(8).plot(kind='barh',logx=True, stacked=False)
show_and_save_plot('Distribucion de eventos en los modelos mas populares', xlabel='Distribucion', ylabel='Modelo')

Modelos de telefono mas vendidos en proporcion a los eventos totales

Vale la pena aclarar que son modelos distintos a los mas populares

In [ ]:
models = models.sort_values([('event','conversion')], ascending=False)
models.event.head(8).plot(kind='barh',logx=True, stacked=False)
plt.xlabel('Distribucion')
plt.ylabel('Modelo')
show_and_save_plot('Distribucion de eventos en los modelos con mejor conversion rate')

In [ ]:
models = models.sort_values([('event','lead')], ascending=False)
models.event.head(8).plot(kind='barh',logx=True, stacked=False)
plt.xlabel('Distribución')
plt.ylabel('Modelo')
show_and_save_plot('Distribucion de eventos en los modelos mas reservados')

## Evaluo cuales son los colores mas vendidos.

In [ ]:
colors_count = raw_events[raw_events.event == 'conversion'].color.dropna().value_counts()

In [ ]:
plt.xlabel('Cantidad')
plt.ylabel('Color')
colors_count.plot(kind = 'barh')
show_and_save_plot('Colores de telefonos mas vendidos', xlabel='Cantidad', ylabel='Color')

## Veo cual es la cantidad de usuarios que ingresaron a la página a través de una campaña publicitaria y terminaron comprando un producto. Para evaluar que tan conveniente es seguir manteniendo cada campaña publicitaria.

In [ ]:
campaign = raw_events[~pd.isna(raw_events["campaign_source"])].drop_duplicates('person')

In [ ]:
campaign["campaign_source"].value_counts().plot(kind = 'barh')

In [ ]:
conversions = raw_events[raw_events['event'] == 'conversion'].drop_duplicates('person')

In [ ]:
pd.merge(conversions, campaign, how = 'inner', on = 'person')['campaign_source_y'].value_counts().plot(kind = 'barh')

In [ ]:
raw_events["search_term"].value_counts()[:10]

## Veamos como se comportan las ventas de teléfonos durante los dias de semana

In [ ]:
raw_events['day_name'] = raw_events['timestamp'].map(lambda x: x.strftime("%A"))


In [ ]:
raw_events['day_number'] = raw_events['timestamp'].map(lambda x: x.weekday())

In [ ]:
# Cambiando 'conversion' por 'visited site' se puede hacer el mismo analisis con la cantidad de visitas.
conversion_by_day = raw_events[raw_events.event == 'conversion']['day_name'].value_counts() 
conversion_by_day = pd.DataFrame(conversion_by_day)

In [ ]:
sorter = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sorterIndex = dict(zip(sorter,range(len(sorter))))

In [ ]:
conversion_by_day['Day_id'] = conversion_by_day.index
conversion_by_day['Day_id'] = conversion_by_day['Day_id'].map(sorterIndex)
conversion_by_day.sort_values('Day_id', inplace = True)

In [ ]:
f, ax = plt.subplots(1)
ax.plot(conversion_by_day.index, conversion_by_day['day_name'], "-o")
ax.set_ylim(ymin=0)

In [ ]:
from datetime import datetime, timedelta

def week_of_month(date):
    month = date.month
    week = 0
    while date.month == month:
        week += 1
        date -= timedelta(days=7)
    return week

raw_events['week_of_month'] = raw_events['timestamp'].apply(week_of_month)

In [ ]:
raw_events['day_number'].value_counts()

In [ ]:
conversions = raw_events[raw_events.event == 'conversion']

In [ ]:
group = conversions.groupby(['day_number', 'week_of_month'])\
        .count().reset_index()\
            [['day_number','week_of_month', 'timestamp']]

In [ ]:
pivot = group.pivot(index = 'week_of_month', columns = 'day_number', values = 'timestamp')

In [ ]:
sns.heatmap(pivot)